In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv
/kaggle/input/house-prices-advanced-regression-techniques/train.csv


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

from sklearn.metrics import accuracy_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [3]:
train_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
combined = [train_df, test_df]
train_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
print(train_df.info())
print('*'*30)
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [5]:
def col_remover(col, ub):
    res = Counter(train_df[col])
    if res.most_common()[0][1]>ub: return True
    return False

cols = []
for col in train_df.columns:
    if col_remover(col, 1100): cols.append(col)
cols.append('Id')
print(len(cols))

for_submission = test_df['Id']
for i in combined:
    i.drop(cols, axis=1, inplace=True)

36


In [6]:
for df in combined:
    for col in train_df.columns:
        if col=='SalePrice': continue
        if Counter(df[col].isnull())[1]<300:
            if df[col].dtypes=='int64': df[col] = df[col].fillna(df[col].dropna().mean())
            elif df[col].dtypes=='float64': df[col] = df[col].fillna(df[col].dropna().mean())
            else: df[col] = df[col].fillna(df[col].dropna().mode()[0])

for df in combined:
        df.drop(['FireplaceQu'], axis=1, inplace=True)

In [7]:
s = 'MSSubClass MSZoning Street Alley LotShape LandContour Utilities LotConfig LandSlope Neighborhood Condition1 BldgType HouseStyle OverallQual OverallCond RoofStyle RoofMatl Exterior1st Exterior2nd MasVnrType ExterQual ExterCond Foundation BsmtQual BsmtCond BsmtExposure BsmtFinType1 BsmtFinType2 Heating HeatingQC CentralAir Electrical KitchenQual Functional FireplaceQu GarageType GarageFinish GarageQual GarageCond PavedDrive PoolQC Fence MiscFeature SaleType SaleCondition'
categorical = s.split()

def get_mapping(df, col):
    temp=dict()
    t=0
    for i in df[col].unique():
        temp[i]=t
        t=t+1
    return temp

def check_mapping(df, col, temp):
    size = len(temp)
    for i in df[col].unique():
        temp[i] = temp.get(i, size)
        if temp[i]==size: size=size+1
    return temp

for col in categorical:
    if col in train_df:
        if col=='SalePrice': continue
        ddict = get_mapping(train_df, col)
        train_df[col] = train_df[col].map(ddict).astype(int)
        ddict = check_mapping(test_df, col, ddict)
        test_df[col] = test_df[col].map(ddict).astype(int)

In [8]:
diff_dtype=[]
for col in train_df:
    if col=='SalePrice': continue
    if train_df[col].dtypes!=test_df[col].dtypes: diff_dtype.append(col)

for col in diff_dtype:
    print(col, train_df[col].dtypes, test_df[col].dtypes)
    
test_df = test_df.astype( {i:int for i in diff_dtype} )

BsmtFinSF1 int64 float64
BsmtUnfSF int64 float64
TotalBsmtSF int64 float64
BsmtFullBath int64 float64
GarageCars int64 float64
GarageArea int64 float64


In [9]:
for df in [train_df, test_df]:
    for col in df:
        if df[col].dtypes=='float64':
            df[col]=df[col].astype(int)

In [10]:
print(train_df.info())
print('*'*30)
print(test_df.info())
print('*'*30)
print([i.shape for i in combined])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 44 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   MSSubClass    1460 non-null   int64
 1   LotFrontage   1460 non-null   int64
 2   LotArea       1460 non-null   int64
 3   LotShape      1460 non-null   int64
 4   LotConfig     1460 non-null   int64
 5   Neighborhood  1460 non-null   int64
 6   HouseStyle    1460 non-null   int64
 7   OverallQual   1460 non-null   int64
 8   OverallCond   1460 non-null   int64
 9   YearBuilt     1460 non-null   int64
 10  YearRemodAdd  1460 non-null   int64
 11  Exterior1st   1460 non-null   int64
 12  Exterior2nd   1460 non-null   int64
 13  MasVnrType    1460 non-null   int64
 14  MasVnrArea    1460 non-null   int64
 15  ExterQual     1460 non-null   int64
 16  Foundation    1460 non-null   int64
 17  BsmtQual      1460 non-null   int64
 18  BsmtExposure  1460 non-null   int64
 19  BsmtFinType1  1460 non-null

In [11]:
cols = 'LotFrontage LotArea YearBuilt YearRemodAdd MasVnrArea BsmtFinSF1 BsmtUnfSF TotalBsmtSF 1stFlrSF 2ndFlrSF GrLivArea GarageYrBlt GarageArea WoodDeckSF OpenPorchSF YrSold'.split()

for df in [train_df, test_df]:
    for col in cols:
        df[col] = pd.cut(df[col], bins=5, labels=range(5))

In [12]:
for df in [train_df, test_df]:
    for col in df:
        df[col] = [int(i) for i in df[col].values]

In [13]:
X = train_df.drop(['SalePrice'], axis=1)
y = train_df['SalePrice']
train_X, val_X, train_y, val_y = train_test_split(X, y)
model_names, acc, err = [],[],[]

In [14]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeRegressor

rf = RandomForestClassifier(random_state=1)
xgb = XGBClassifier()
lr = LogisticRegression(solver='saga')
mlp = MLPClassifier()
dtc = DecisionTreeRegressor(random_state=0)

model_names = ['rf', 'xgb', 'lr', 'mlp', 'dtc']
models = [rf, xgb, lr, mlp, dtc]

In [15]:
for i in range(len(models)):
    models[i].fit(train_X, train_y)
    prediction = models[i].predict(val_X)
    acc.append(accuracy_score(val_y, prediction))
    err.append(mean_absolute_error(val_y, prediction))

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [16]:
models = pd.DataFrame({
    'Model': [i.upper() for i in model_names],
    'Accuracy': acc,
    'Mean Abs Error': err})
models.sort_values(by='Accuracy', ascending=False)

,Model,Accuracy,Mean Abs Error
0,RF,0.016438,26043.947945
3,MLP,0.008219,30699.695890
2,LR,0.005479,32619.786301
1,XGB,0.002740,32187.276712
4,DTC,0.000000,29846.991781


In [17]:
final_rf = RandomForestClassifier(random_state=1)
final_xgb = XGBClassifier()
final_lr = LogisticRegression(solver='saga')
final_mlp = MLPClassifier()
final_dtc = DecisionTreeRegressor(random_state=0)

models = [final_rf, final_xgb, final_lr, final_mlp, final_dtc]
for i in range(len(models)):
    models[i].fit(X,y)
    vals = models[i].predict(test_df)
    file = pd.DataFrame({'Id':for_submission, 'SalePrice':vals})
    file.to_csv('submission_'+model_names[i].upper()+'.csv', index = False)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
